# Robot on a 2-D plane

In [1]:
from sympy import *
from sympy.physics.mechanics import dynamicsymbols
init_printing()

## Let's define a system in the form:
\begin{align}
\dot{x} &= A x + B u \\
y &= C^T x\\
\end{align}

## With the states:
\begin{align}
x &= [x_1,x_2,x_3,x_4,x_5,x_6]^T
\end{align}

### where:

$x_1$ - distance in x on the plane from origin

$x_2$ - distance in y on the plane from origin

$x_3$ - speed in relation to the robot's x direction

$x_4$ - acceleration in relation to the robot's x direction

$x_5$ - rotation around the robot's z-axis

$x_6$ - angular velocity around the robot's z-axis

In [2]:
t = Symbol("t")
x1 = dynamicsymbols("x1")
x2 = dynamicsymbols("x2")
x3 = dynamicsymbols("x3")
x4 = x3.diff(t)
x5 = dynamicsymbols("x5")
x6 = x5.diff(t)

## With the input:
\begin{align}
u &= [F_x,\tau_z]^T
\end{align}

### where:

$F_x$ - Force in the robot's relative x direction

$\tau_z$ - Moment around the robot's z-axis

In [3]:
F_x = Symbol("F_x")
tau_z = Symbol("tau_z")

## With the output:
\begin{align}
y &= [a_{accel},\dot{\psi}_{gyro,z}]^T
\end{align}

### where:

$a_{accel,x}$ - acceleration measured in the x direction by the accelerometer

$\dot{\psi}_{gyro,z}$ - angular velocity measured by the gyroscope on the robot

In [4]:
a_accelx = Symbol("a_accelx")
dpsi_gyroz = Symbol("dpsi_gyroz")

## We can define the B matrix as:
\begin{align}
B &=
\begin{bmatrix}
0 & 0 \\
0 & 0 \\
0 & 0 \\
\frac{1}{m} & 0 \\
0 & 0 \\
0 & \frac{1}{J}
\end{bmatrix}
\end{align}

## We can define the C matrix as:
\begin{align}
C^T &= 
\begin{bmatrix}
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}
\end{align}

In [5]:
C = Matrix([[0,0,0,0,1,0,0],[0,0,0,0,0,0,1]]).T
pprint(C)

⎡0  0⎤
⎢    ⎥
⎢0  0⎥
⎢    ⎥
⎢0  0⎥
⎢    ⎥
⎢0  0⎥
⎢    ⎥
⎢1  0⎥
⎢    ⎥
⎢0  0⎥
⎢    ⎥
⎣0  1⎦


## Define rotation matrix $R_{z,\psi}$ around axis z:

In [6]:
R_z_psi = rot_axis1(x5).T
pprint(R_z_psi)

⎡1      0            0     ⎤
⎢                          ⎥
⎢0  cos(x₅(t))  -sin(x₅(t))⎥
⎢                          ⎥
⎣0  sin(x₅(t))  cos(x₅(t)) ⎦


## Create rotation velocity matrix $S_{z,\psi}$ with: 
\begin{align}
S_{z,\psi} = \dot{R}_{z,\psi} R_{z,\psi}^T
\end{align}

In [7]:
S_z_psi = R_z_psi.diff(t)*R_z_psi.T
pprint(simplify(S_z_psi))

⎡0      0          0     ⎤
⎢                        ⎥
⎢               d        ⎥
⎢0      0      -──(x₅(t))⎥
⎢               dt       ⎥
⎢                        ⎥
⎢   d                    ⎥
⎢0  ──(x₅(t))      0     ⎥
⎣   dt                   ⎦


## We can describe the rotation as:
\begin{align}
\begin{bmatrix}
x_0 \\
x_1 \\
\end{bmatrix} &=
R_{z,\psi}(x_5)
\begin{bmatrix}
x_3 t\\
0 \\
0\\
\end{bmatrix} +
R_{z,\psi}(x_5)
\begin{bmatrix}
\frac{1}{2} x_4 t^2\\
0 \\
0\\
\end{bmatrix}
\end{align}

### with the origin: $[0, 0]^T$

## With the transformation:
\begin{align}
H(x_5) &= 
\begin{bmatrix}
R_{z,\psi}(x_5) & 0 \\
0 & 1 \\
\end{bmatrix}
\end{align}

## A matrix?

\begin{align}
\begin{bmatrix}
\dot{x}_1 \\
\dot{x}_2 \\
\dot{x}_3 \\
\dot{x}_4 \\
\dot{x}_5 \\
\dot{x}_6 \\
\end{bmatrix} &=
\begin{bmatrix}
1 & 0 & ? & ? & 0 & 0\\
1 & 0 & ? & ? & 0 & 0\\
0 & 0 & 1 & t & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & t\\
0 & 0 & 0 & 0 & 0 & 1\\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\
x_6 \\
\end{bmatrix}
\end{align}


How do you construct the A matrix so its independent from $x_5$?